In [1]:
import time
import math

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import precision_recall_fscore_support

from tqdm.notebook import tqdm

# Пункт 1 и 2

In [2]:
df = pd.read_csv("../assets/annotated-corpus/test-embeddings.tsv", sep="\t", header=None, index_col=False)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11911 entries, 0 to 11910
Columns: 101 entries, 0 to 100
dtypes: float64(100), object(1)
memory usage: 9.2+ MB


In [4]:
df["target"] = df[0].str.rsplit("_", n=1, expand=True)[0]
df.head()

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,target
0,age_3351,-0.268966,-0.195137,0.185221,0.442006,-1.399686,0.458122,0.090845,-0.372571,-0.536477,...,-0.228859,0.308409,0.602842,0.037475,-0.383144,-0.151171,0.243271,0.118566,-0.137218,age
1,age_11616,-0.540151,0.308859,0.031820,0.343899,-0.737027,0.908944,-0.476058,-0.001940,-0.186527,...,-0.021206,0.141861,-0.207191,0.167495,0.345437,-0.090632,0.256034,0.357400,0.161623,age
2,age_1546,-0.277256,-0.396026,0.232861,0.595492,-1.076954,0.648078,0.039423,-0.139098,-0.631634,...,-0.121855,-0.362199,0.621111,-0.396075,-0.874889,-0.389040,0.304413,0.077354,-0.220831,age
3,age_6229,0.269636,-0.933570,0.436701,0.543866,-1.554088,0.582454,0.190633,-0.799669,-0.566312,...,-0.397914,1.153079,0.111551,0.063922,-1.197466,0.065416,0.369039,0.540426,0.092124,age
4,age_8562,-0.684108,-1.384038,0.142103,0.670762,-1.951569,0.886495,-0.431131,-0.198965,-0.063547,...,-0.449400,-0.210606,-0.299428,-0.684055,-0.388931,-0.294863,0.478414,-0.463894,0.867589,age


In [5]:
label_encoder = LabelEncoder()
df["target_enc"] = label_encoder.fit_transform(df["target"])

In [6]:
df[["target", "target_enc"]]

,target,target_enc
0,age,0
1,age,0
2,age,0
3,age,0
4,age,0
...,...,...
11906,not_cyberbullying,3
11907,not_cyberbullying,3
11908,not_cyberbullying,3
11909,not_cyberbullying,3


In [7]:
df["target"].unique(), df["target_enc"].unique()

(array(['age', 'ethnicity', 'gender', 'religion', 'other_cyberbullying',
        'not_cyberbullying'], dtype=object),
 array([0, 1, 2, 5, 4, 3]))

In [9]:
def confusion_matrix(true, pred):
    classes = set(true + pred)
    num_classes = len(classes)
    mat = np.zeros((num_classes, num_classes))
    n = max(len(true), len(pred))
    for i in range(num_classes):
        for j in range(num_classes):
            for k in range(n):
                if true[k] == i:
                    if pred[k] == j:
                        mat[i][j] = mat[i][j] + 1
    return mat

In [10]:
def get_precision_recall_fscore_accuracy(cm, beta=1.0):
    true_pos = np.diag(cm)
    false_pos = np.sum(cm, axis=0) - true_pos
    false_neg = np.sum(cm, axis=1) - true_pos
    
    precision = true_pos / (true_pos + false_pos)
    recall = true_pos / (true_pos + false_neg)

    numerator = (1 + math.pow(beta, 2)) * recall * precision
    denominator = (math.pow(beta, 2) * precision) + recall

    fscore = numerator / denominator

    accuracy = true_pos / np.sum(cm, axis=1)

    return precision, recall, fscore, accuracy

In [11]:
def recall_precision(matrix_df, level = 'micro'):
    arr = matrix_df.to_numpy() # Total Number of Instance

    rows = np.sum(arr, axis = 1) # Sum of rows of each class (TP(i) + FN(i))
    columns = np.sum(arr, axis = 0) # Sum of columns of each class (TP(i) + FP(i))

    diagonals = np.diag(arr) # Get the diagonals 

    if (level == 'micro'):
        # sum of TP(i) / sum of (TP(i) + FN(i))
        recall = sum(diagonals) * 100 / sum(rows)
        # sum of TP(i) / sum of (TP(i) + FP(i))
        precision = sum(diagonals) * 100 / sum(columns)
    elif (level == 'macro'):
        # sum of recall(i) / c
        recall = sum((diagonals / rows)) * 100 / len(diagonals)
        # sum of precision(i) / c
        precision = sum((diagonals / columns)) * 100 / len(diagonals)
    elif (level == 'weighted'):
        # sum of recall(i) * true proportion of the class
        recall = sum((diagonals / rows) * (rows / np.sum(arr))) * 100
        # sum of precision(i) * true proportion of the class
        precision = sum((diagonals / columns) * (rows / np.sum(arr))) * 100

    return recall, precision


def compute_f_score(recall, precision, beta = 1.0):
    numerator = (1 + math.pow(beta, 2)) * recall * precision
    denominator = (math.pow(beta, 2) * precision) + recall

    return numerator/denominator


def compute_accuracy(matrix_df, predictions):
    accuracy = matrix_df.to_numpy().trace() * 100 / len(predictions)
    return accuracy


def get_precision_recall_fscore_accuracy_v2(cm, level="macro", beta=1.0):
    recall, precision = recall_precision(cm, level)
    

In [18]:
params = [
    { 
        "kernel": ["linear"],
        "C": [0.75, 1.0, 1.25],
        "gamma": ["scale", "auto"]
    },
    { 
        "kernel": ["poly"],
        "degree": [3],
        "C": [1.0, 1.25, 1.5],
        "gamma": ["scale", "auto"]
        # "class_weight": [None, "balanced"]
    },
    {
        "kernel": ["rbf"],
        "C": [1.0, 1.25, 1.5],
        "gamma": ["scale", "auto"]
        # "class_weight": [None, "balanced"]
    },
    {
        "kernel": ["sigmoid"],
        "C": [0.5, 0.75, 1.0],
        "gamma": ["scale", "auto"]
    }
]

param_grid = ParameterGrid(params)

In [19]:
X = df[df.columns.difference([0, 'target', 'target_enc'])]
y = df["target_enc"]

In [20]:
metrics = {
    "accuracy": dict(),
    "precision": dict(),
    "recall": dict(),
    "fscore" : dict(),
    "exec_time": dict()
}

metrics_names = ["accuracy", "precision", "recall", "fscore", "exec_time"]
for i, param in tqdm(enumerate(param_grid)):
    clf = SVC(**param)
    
    start_time = time.time()
    clf.fit(X, y)
    exec_time = time.time() - start_time
    
    y_pred = clf.predict(X)

    cm = confusion_matrix(y.tolist(), y_pred.tolist())
    pr, rec, fscore, acc = get_precision_recall_fscore_accuracy(cm)

    print(f"Model version №{i + 1}")
    print("params", param)
    for metr, name in zip([acc, pr, rec, fscore, exec_time], metrics_names):
        metrics[name][f"model_{i + 1}"] = metr
        print(name, np.mean(metr))

0it [00:00, ?it/s]

/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be

Model version №1
params {'C': 0.75, 'gamma': 'scale', 'kernel': 'linear'}
accuracy 0.7575182907689778
precision 0.7578145068863561
recall 0.7575182907689778
fscore 0.7564940982370721
exec_time 5.318994998931885


/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be

Model version №2
params {'C': 0.75, 'gamma': 'auto', 'kernel': 'linear'}
accuracy 0.7575182907689778
precision 0.7578145068863561
recall 0.7575182907689778
fscore 0.7564940982370721
exec_time 5.4707677364349365


...

Model version №22
params {'C': 0.75, 'gamma': 'auto', 'kernel': 'sigmoid'}
accuracy 0.697974335133401
precision 0.6978018060832382
recall 0.697974335133401
fscore 0.697674053720097
exec_time 3.975247859954834


/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be

Model version №23
params {'C': 1.0, 'gamma': 'scale', 'kernel': 'sigmoid'}
accuracy 0.6352340566259573
precision 0.6468693849345032
recall 0.6352340566259573
fscore 0.6385478336222832
exec_time 3.004000425338745


/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be

Model version №24
params {'C': 1.0, 'gamma': 'auto', 'kernel': 'sigmoid'}
accuracy 0.6913751677767334
precision 0.6899422174913368
recall 0.6913751677767334
fscore 0.6905091728505265
exec_time 3.677698850631714


In [30]:
def grid_search(X, y, param_grid):
    metrics = {
        "accuracy": dict(),
        "precision": dict(),
        "recall": dict(),
        "fscore" : dict(),
        "exec_time": dict()
    }

    model_params = dict()
    
    metrics_names = ["accuracy", "precision", "recall", "fscore", "exec_time"]
    for i, param in tqdm(enumerate(param_grid)):
        clf = SVC(**param)
        
        start_time = time.time()
        clf.fit(X, y)
        exec_time = time.time() - start_time
        
        y_pred = clf.predict(X)
    
        cm = confusion_matrix(y.tolist(), y_pred.tolist())
        pr, rec, fscore, acc = get_precision_recall_fscore_accuracy(cm)
    
        print(f"Model version №{i + 1}")
        print("params", param)
        for metr, name in zip([acc, pr, rec, fscore, exec_time], metrics_names):
            metrics[name][f"model_{i + 1}"] = metr
            print(name, np.mean(metr))

        model_params[f"model_{i + 1}"] = param

    return metrics, model_params
    

In [28]:
def find_best_model_by_metrics(metric_model, metrics_names):
    for name in metrics_names:
        k, v = max(metric_model[name].items(), key=lambda x: np.mean(x[1]))
        print(f"Metric {name}: model {k} with mean value {np.mean(v)}")
    

In [31]:
metrics_1, model_params_1 = grid_search(X, y, param_grid)

0it [00:00, ?it/s]

/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be

Model version №1
params {'C': 0.75, 'gamma': 'scale', 'kernel': 'linear'}
accuracy 0.7575182907689778
precision 0.7578145068863561
recall 0.7575182907689778
fscore 0.7564940982370721
exec_time 5.8660407066345215


/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be

Model version №2
params {'C': 0.75, 'gamma': 'auto', 'kernel': 'linear'}
accuracy 0.7575182907689778
precision 0.7578145068863561
recall 0.7575182907689778
fscore 0.7564940982370721
exec_time 5.709137201309204


...

Model version №22
params {'C': 0.75, 'gamma': 'auto', 'kernel': 'sigmoid'}
accuracy 0.697974335133401
precision 0.6978018060832382
recall 0.697974335133401
fscore 0.697674053720097
exec_time 3.8829052448272705


/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be

Model version №23
params {'C': 1.0, 'gamma': 'scale', 'kernel': 'sigmoid'}
accuracy 0.6352340566259573
precision 0.6468693849345032
recall 0.6352340566259573
fscore 0.6385478336222832
exec_time 2.89493989944458


/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be

Model version №24
params {'C': 1.0, 'gamma': 'auto', 'kernel': 'sigmoid'}
accuracy 0.6913751677767334
precision 0.6899422174913368
recall 0.6913751677767334
fscore 0.6905091728505265
exec_time 3.6250030994415283


In [35]:
find_best_model_by_metrics(metrics_1, ["accuracy", "precision", "recall", "fscore"])

Metric accuracy: model model_17 with mean value 0.8333432147696346
Metric precision: model model_17 with mean value 0.8361782536584718
Metric recall: model model_17 with mean value 0.8333432147696346
Metric fscore: model model_17 with mean value 0.832782672012491


In [38]:
best_model_metrics = { metric_name: val for metric_name, d in metrics.items() for model_name, val in d.items() if model_name == "model_17" }

In [61]:
best_model_metrics

{'accuracy': array([0.93953252, 0.95639247, 0.78165711, 0.621661  , 0.76255708,
        0.93825911]),
 'precision': array([0.90415648, 0.94654242, 0.89659295, 0.70874862, 0.64952463,
        0.91150442]),
 'recall': array([0.93953252, 0.95639247, 0.78165711, 0.621661  , 0.76255708,
        0.93825911]),
 'fscore': array([0.92150511, 0.95144195, 0.83518931, 0.66235446, 0.70151692,
        0.92468828]),
 'exec_time': 4.008533954620361}

# Пункт 3

In [48]:
X_log = X.copy().apply(np.log).fillna(0.0)
X_sin = X.copy().apply(np.sin)

In [52]:
metrics_names = ["accuracy", "precision", "recall", "fscore", "exec_time"]

In [56]:
def fit_predict(X, y, model_params):
    clf = SVC(**model_params_1["model_17"])
    start_time = time.time()
    clf.fit(X, y)
    exec_time = time.time() - start_time
    
    y_pred = clf.predict(X)
    cm = confusion_matrix(y.tolist(), y_pred.tolist())
    pr, rec, fscore, acc = get_precision_recall_fscore_accuracy(cm)
    
    metrics = dict()
    for metr, name in zip([acc, pr, rec, fscore, exec_time], metrics_names):
        metrics[name] = metr
        print(name, np.mean(metr))

    return metrics

In [58]:
log_model_metrics = fit_predict(X_log, y, model_params_1["model_17"])

/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be

accuracy 0.8821543768388674
precision 0.8825037814110784
recall 0.8821543768388674
fscore 0.8816307546242369
exec_time 9.05287504196167


In [59]:
sin_model_metrics = fit_predict(X_sin, y, model_params_1["model_17"])

/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:595: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:604: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Applications/anaconda3/envs/py310/lib/python3.10/site-packages/sklearn/utils/validation.py:757: FutureWarning: is_sparse is deprecated and will be

accuracy 0.875356590941268
precision 0.8778817492599628
recall 0.875356590941268
fscore 0.8750483402065377
exec_time 3.8480849266052246
